# Example of the usage of the BigDFT routines from a Jupyter notebook
This notebook will explain, from the examples shown [here](http://nbviewer.jupyter.org/github/mgaitan/fortran_magic/blob/master/documentation.ipynb) what are the main passages to utilize the BigDFT routines from within a Jupyter environment. We will provide as an example the usage of some routines provided in the BigDFT modules.

Let us first load the fortran_magic package as explained in the above referenced notebook.

In [1]:
%load_ext fortranmagic

/usr/local/lib/python3.4/dist-packages/fortranmagic.py:147: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module since IPython 4.0.
  self._lib_dir = os.path.join(get_ipython_cache_dir(), 'fortran')


In [2]:
%fortran_config

Current defaults arguments for %fortran:
	--extra '-I/local/binaries/gfortran-bindings-fpe/install/include  -L/local/binaries/gfortran-bindings-fpe/install/lib -lbigdft-1 -lbabel -lPSolver-1 -lfutile-1'


Then we can start the reference example to control that our implemenation is apparently functional:

In [3]:
%%fortran

subroutine f1(x, y, z)
    real, intent(in) :: x,y
    real, intent(out) :: z

    z = sin(x+y)

end subroutine f1

In [4]:
import numpy as np
print(f1(np.pi,0.0))
print(f1(0.5*np.pi,0.5*np.pi))
print(f1(0.25*np.pi,0.25*np.pi))

-8.742277657347586e-08
-8.742277657347586e-08
1.0


The above numbers should be two (simple precision) zeros and a one.

Let us now write a routine that import a integer into a futile dictionary; this routine is an example on how we might use the futile module within fortran_magic.

First, let us retrieve the link line for futile and bigdft:

In [5]:
!$FUTILE_SOURCES/../Installer.py link futile -yq

Parsing arguments with method argparse...
Configuration chosen for the Installer:
  Hostname: athelas
  Source directory: /local/genovese/work/BigDFT/1.8
  Compiling from a branch: True
  Build directory: /local/genovese/work/execute/BigDFT-nb/tutorial
  Action chosen: link
  Verbose: False
  Jhbuild baseline: /local/genovese/work/BigDFT/1.8/futile/../jhbuild.py 
 List of modules to be treated: ['libyaml', 'futile']
--------- Linking line to build with package "futile":
  -I/local/binaries/gfortran-bindings-fpe/install/include  -L/local/binaries/gfortran-bindings-fpe/install/lib -lfutile-1  
--------------------------------------------------
Thank you for using the Installer of BigDFT suite.
The action considered was: link


In [6]:
%fortran_config --extra '-I/local/binaries/gfortran-bindings-fpe/install/include  -L/local/binaries/gfortran-bindings-fpe/install/lib -lfutile-1'

New default arguments for %fortran:
	--extra '-I/local/binaries/gfortran-bindings-fpe/install/include  -L/local/binaries/gfortran-bindings-fpe/install/lib -lfutile-1'


In [7]:
%%fortran
subroutine test_dict(iin,iout)
use futile
integer, intent(in) :: iin
integer, intent(out) :: iout
!local variables
type(dictionary), pointer :: test
        
call f_lib_initialize() !initalize futile
        
test=>dict_new() !create a dictionary object
call set(test//'integer',iin) !associate an integer to such a dictionary

iout=test//'integer' !retrieve the value

call f_increment(iout) !iout += 1

call dict_free(test)
call f_lib_finalize_noreport()
end subroutine test_dict

In [8]:
test_dict(3)

4

Cool, it works! We can use with this scheme a deeper routine of BigDFT.
For example, we can

* Read a file containing atomic positions
* Construct a astruct from the data of this file
* Identify from this structure the nearest neighbor of each of the atoms
* Store them in a matrix of the size of the atomic positions

In [15]:
%%fortran -vvv
subroutine nearest_neighbor(file,n,nn_list)
use module_atoms
use futile
use f_trees
implicit none
character(len=*), intent(in) :: file
integer, intent(in) :: n
real(8), intent(out), dimension(15,n) :: nn_list
!local variables
type(f_tree) :: tree_posinp
type(atomic_structure) :: astruct

!load the atomic position in the tree_posinp
tree_posinp=f_tree_load(file)

!> fill the atomic structure datatype
call astruct_set(astruct,tree_posinp%d,0.0_f_double,.true.,1.e-8_f_double,&
  [0.0_f_double,0.0_f_double,0.0_f_double],1,.true.)

end subroutine

Running...
   /usr/bin/python3 -m numpy.f2py -I/local/binaries/gfortran-bindings-fpe/install/include -Wl,-rpath=/local/binaries/gfortran-bindings-fpe/install/lib -L/local/binaries/gfortran-bindings-fpe/install/lib -lbigdft-1 -lbabel -lPSolver-1 -lfutile-1 -I/local/binaries/gfortran-bindings-fpe/install/include -Wl,-rpath=/local/binaries/gfortran-bindings-fpe/install/lib -L/local/binaries/gfortran-bindings-fpe/install/lib -lbigdft-1 -lbabel -lPSolver-1 -lfutile-1 -I/local/binaries/gfortran-bindings-fpe/install/include -L/local/binaries/gfortran-bindings-fpe/install/lib -lbigdft-1 -lbabel -lPSolver-1 -lfutile-1 -I/local/binaries/gfortran-bindings-fpe/install/include -L/local/binaries/gfortran-bindings-fpe/install/lib -lbigdft-1 -lbabel -lPSolver-1 -lfutile-1 -I/local/binaries/gfortran-bindings-fpe/install/include -L/local/binaries/gfortran-bindings-fpe/install/lib -lbigdft-1 -lbabel -lPSolver-1 -lfutile-1 -I/local/binaries/gfortran-bindings-fpe/install/include -L/local/binaries/gfortran-

error: unknown file type '' (from '-Wl,-rpath=/local/binaries/gfortran-bindings-fpe/install/lib')


RuntimeError: f2py failed, see output